In [65]:
from pathlib import Path
import pandas as pd
import logging
from tqdm import tqdm

tqdm.pandas()

logger = logging.getLogger()

### Load Model and Tokenizer

In [75]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration

MODEL_CKPT = 'paust/pko-t5-larget'
MODEL_CKPT = 'paust/pko-t5-base'

tokenizer = T5TokenizerFast.from_pretrained(MODEL_CKPT)
model = T5ForConditionalGeneration.from_pretrained(MODEL_CKPT)

print(MODEL_CKPT)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


paust/pko-t5-base


### Load Data

In [66]:
def _load_json_data(data_file_dir: Path) -> pd.DataFrame:
    if data_file_dir.exists():
        df = pd.read_json(data_file_dir, encoding="utf-8", orient="records", lines=True)
    else:
        logger.info(f"{data_file_dir} is not exists, return empty ")
        df = pd.DataFrame()
    return df

In [67]:
train_data1 = Path("./data/news_topic_trainset1.json")
valid_data1 = Path("./data/news_topic_validset1.json")
train_data2 = Path("./data/news_topic_trainset2.json")
valid_data2 = Path("./data/news_topic_validset2.json")

### v1 and v2 Comparison

In [133]:
df_v1 = pd.concat([_load_json_data(train_data1), _load_json_data(valid_data1)]).reset_index(drop=True)
df_v2 = pd.concat([_load_json_data(train_data2), _load_json_data(valid_data2)]).reset_index(drop=True)

df_v1.rename(columns={"prompt": "input_text", "completion": "target_text"}, inplace=True)
df_v2.rename(columns={"prompt": "input_text", "completion": "target_text"}, inplace=True)

In [136]:
df_v1.target_text.progress_apply(lambda x: len(tokenizer(x).input_ids)).describe()

100%|██████████| 928/928 [00:00<00:00, 13718.33it/s]


count    928.000000
mean      18.560345
std        7.814780
min        4.000000
25%       13.000000
50%       17.000000
75%       22.000000
max       88.000000
Name: target_text, dtype: float64

In [139]:
for idx, row in df_v1.iterrows():
    # print(row.input_text)
    print(row.target_text)
    print()

 주택담보대출,ltv 규제 완화,dsr


 송혜교,장기용,사랑,애절함,지헤중,최고 시청률,민여사,강정자,윤수완,신유정


 자가검사키트


 돌싱글즈,유세윤,부모님 이혼,해외 출장,빈하영,정윤식


 가격 인상,휘발유 가격


 코로나19,ad카드,관광,은메달리스트,긴급사태,도쿄올림픽


 이민정,이병헌


 김영찬,기부,버디기금,대전사회복지공동모금회,이웃사랑


 우아한형제들,근무장소,시간 자율 전환,근무지 자율선택제


 연애도사,제이쓴,홍현희


 손해배상


 뚜레쥬르 제일제당센터점,매출,방문객,제품,공간,리뉴얼


 충북 진천 아파트,화재사고


 최사라·한상민·황민규,2022베이징패럴림픽 파라 알파인스키종목 출전


 앞으로 2주간 모든 내·외국인 입국자,예방접종 관계없이 10일간 격리,나이지리아 방역강화국가·위험국,코로나19


 war,ukraine,austria


 한국프로축구연맹,유소년 성 문제,징계 절차


 서울스카이,미디어체험전,시간,하늘,광고,패션,한영수,mz세대,사진,전시회,체험형


 오징어 게임,놀면 뭐하니,럭비 국가대표


 핫이슈,뮤직 엑세스,라디오 캡처,매력,신인,그라타타,비하인드스토리,스나이퍼,안무,가창력,버킷리스트


 엘앤케이바이오,척추수술,권위자,앤서니 신,협력


 오정근,남기선,김애란,정경화,임준성


 맨유,리그 선두 도전


 조정석,전미도,삼시세끼,무한굴레,슬기로운 산촌생활,공식포스터,라이프,찐친,웃음,따스


 유현철,옥순,악플,전쟁,댓글,인성,돌싱,sbs plus,ena play,mbn,돌싱글즈3,열애


 26살,월수입,300만원,집 포기,1억짜리 차


 sk하이닉스,정기 임원인사,승진,신규 임원,곽노정,노종원,김규현,김상훈,김진영,김헌규


 화천대유,김만배,이재명,행정,정책,뇌물,횡령,배임,수사,특혜,


 총무원장 임기 시작 축하,진우 스님


 잉글랜드-이탈리아 유로2020 결승전,잉글랜드,이탈리아


 이선빈,남친,이광수,유재석


 우루과이,엘 파이스,한국,2002년,판정,4강


 안전

In [144]:
text = '전북도,경기도,자동차 대체 인증부품 산업,활성화,조례 제정,협약 체결,순정품 대비 저렴한 제품,생산·유통·소비 지원,제도개선·정책사업 기획,소비 확대 홍보,송하진 지사,이재명 지사,판로 개척,소비자 선택권,공정경제'
first_ten = ','.join(text.split(',')[:10])
len(tokenizer.encode(first_ten))

60

In [137]:
df_v2.target_text.progress_apply(lambda x: len(tokenizer(x).input_ids)).describe()

100%|██████████| 1585/1585 [00:00<00:00, 10369.99it/s]


count    1585.000000
mean       32.882019
std        27.917593
min         5.000000
25%        15.000000
50%        21.000000
75%        40.000000
max       157.000000
Name: target_text, dtype: float64

In [140]:
for idx, row in df_v2.iterrows():
    # print(row.input_text)
    print(row.target_text)
    print()

배꼽마을,가을단풍,옥천군,중심점


,농협은행,ESG경영,상생경영,제1회 대한민국 리딩금융 ESG 어워드,친환경경영,지배구조,녹색금융,


요소수 ,국회운영위


 노지훈,장모,응급실,살림하는 남자들 시즌2


정의용 외교부 장관,러시아 출국


출산 정책,헝가리,출산정책,인구감소문제


북한은 지금,농촌 살림집 건설


사망전 기관지염,카타르 월드컵,동성애 시위,


한미재계회의 환영만찬,김진표 국회의장,허창수 전경련 회장


스토킹 피해,유튜버 릴카


최대 실적,주가 상향,삼성엔지니어링


충북,먹깨비,정체기,대대적인 홍보,돌파구,배달앱,활성화 지원사업,수수료,소상인들,가맹점,가입자,주문 건수,상승세,홍보 마케팅,지속 가능한 유통 플랫폼


생활가전,역대 최고,LG전자,매출,영업이익,생활가전


중국,손준호,중국슈퍼리그,프리미어리그,유시쯔쉰왕,EPL ,리그랭킹 하락


중소기업벤처기업부,중기부,사업연계형 기술개발 지원사업,중소기업


이장호씨,빙부상


맞춤형 콘텐츠,세라젬,리서치,부모님 연구소


LALISA',스우파,리정 지효 이삭,MV 출연,YGX,블랙핑크 리사


티앤엘,결산배당,주당 400원


송파 추월서울,서초구,반포·잠원 아파트 ,반포·잠원동,강남3구,토지거래허가구역,신고가


한국형 스마트팜 온실 방문,이호승 청와대 정책실장


김병수 감독,인사,강원 춘천시,송암스포츠타운 경기장,울산 현대,강원 FC 경기


음원,록밴드,발매,유튜브,블루바이닐,밴드 더발룬티어스,백예린,


카카오,역성장,영업이익,실적 쇼크,기업지배구조,글로벌 경기 침체,어닝쇼크


윤석열,국회사진기자단,원희룡,유승민,홍준표,토론회


fine-dust,housing,inflation,economy,energy,nuclear energy,pollution


특수본,기성용 부친,농지법 위반,기성용,기영옥,광주경찰청,농업경영계획서,


교육부,전면 등교 준비,코로나19 방역,대통령 직속 국가교육회의,국민 참여 교육과정 개정,사회적 협의 추진계획,온라인 설문조사,권역별 토론회

### Load All

In [69]:
df_train = pd.concat([_load_json_data(train_data1), _load_json_data(train_data2)])
df_valid = pd.concat([_load_json_data(valid_data1), _load_json_data(valid_data2)])

In [70]:
for idx, row in df_train.iterrows():
    if idx == 10:
        break
    print(f'Row {idx}')
    print('input')
    print(row.prompt)
    print()
    print('output')
    print(row.completion)
    print()
    print()

Row 0
input
다음 달부터 15억 초과 주택 대출 허용…ltv 50%도
다음 달부터는 무주택자의 경우 규제 지역에서도 주택담보대출비율(ltv)이 50%로 일괄 적용되고, 투기과열지구의 15억 원이 넘는 아파트에 대한 주택담보대출도 허용됩니다.
금융위원회는 최근 이 같은 내용의 대출 규제 완화 방침을 포함한 개정 은행업 감독규정을 고시하고 다음 달 1일부터 시행에 들어간다고 밝혔습니다.
앞서 정부는 지난달 제11차 비상경제민생회의에서 부동산 규제 완화 방안을 발표하고, 이달 10일 제3차 부동산 관계장관회의에서는 시행 시기를 내년 초에서 연내로 앞당겼습니다.
이를 통해 먼저 현재 규제지역에서 주택 가격에 따라 20∼50%로 차등 적용되고 있는 무주택자 대상 ltv 규제는 50%로 일원화되고, 아파트 가격 제한도 없어지게 됩니다.
현재 비규제지역에서는 무주택자를 대상으로 ltv 70%가 적용되고 있지만, 투기과열지구에서는 무주택자라도 9억 원 이하 주택은 40%, 9억 원 초과 주택은 20%의 ltv를 적용받습니다.
다만 규제지역 내 다주택자를 상대로 신규 주택대출을 금지한 규제는 지금처럼 유지됩니다.
또 총부채원리금상환비율(dsr)에 대한 규제(40%) 역시 그대로 적용됩니다.

###



output
 주택담보대출,ltv 규제 완화,dsr



Row 1
input
송혜교x장기용 "끝까지 사랑일 거야"
..애절함 폭발 '지헤중', 순간 최고 9% [종합]
(엑스포츠뉴스 조혜진 기자) '지헤중' 송혜교와 장기용의 애절 엔딩이 뜨거운 반응을 얻고 있다.
11일 방송된 sbs 금토드라마 '지금, 헤어지는 중입니다'(극본 제인, 연출 이길복, 이하 '지헤중')
10회에서는 하영은(송혜교 분)과 윤재국(장기용)이 더욱 가혹해진 시련과 마주하게 됐다.
두 사람 모두 큰 상처를 받았지만 자신보다 상대를 더 걱정하고 가슴 아파했다.
아픈 만큼 서로를 향한 사랑도, 애절함도 커진 '지헤중' 10회는 순간 최고 시청률 9%를 기록했다. (닐슨코리아 제공)
앞서 윤수완(

### Preprocess

In [71]:
df_train.rename(columns={"prompt": "input_text", "completion": "target_text"}, inplace=True)
df_valid.rename(columns={"prompt": "input_text", "completion": "target_text"}, inplace=True)

df_train["input_text"] = df_train["input_text"].progress_apply(lambda row: row.strip().replace("###", "").strip())
df_valid["input_text"] = df_valid["input_text"].progress_apply(lambda row: row.strip().replace("###", "").strip())

df_train["input_text"] = df_train["input_text"].str.replace('\n', ' ')
df_valid["input_text"] = df_valid["input_text"].str.replace('\n', ' ')

df_train["target_text"] = df_train["target_text"].progress_apply(lambda row: ",".join([sentence.strip() for sentence in row.strip().split(",")]))
df_valid["target_text"] = df_valid["target_text"].progress_apply(lambda row: ",".join([sentence.strip() for sentence in row.strip().split(",")]))

100%|██████████| 825/825 [00:00<00:00, 168171.70it/s]


In [72]:
for idx, row in df_train.iterrows():
    if idx == 10:
        break
    print(f'Row {idx}')
    print('input_text')
    print(row.input_text)
    print()
    print('target_text')
    print(row.target_text)
    print()
    print()

Row 0
input_text
다음 달부터 15억 초과 주택 대출 허용…ltv 50%도 다음 달부터는 무주택자의 경우 규제 지역에서도 주택담보대출비율(ltv)이 50%로 일괄 적용되고, 투기과열지구의 15억 원이 넘는 아파트에 대한 주택담보대출도 허용됩니다. 금융위원회는 최근 이 같은 내용의 대출 규제 완화 방침을 포함한 개정 은행업 감독규정을 고시하고 다음 달 1일부터 시행에 들어간다고 밝혔습니다. 앞서 정부는 지난달 제11차 비상경제민생회의에서 부동산 규제 완화 방안을 발표하고, 이달 10일 제3차 부동산 관계장관회의에서는 시행 시기를 내년 초에서 연내로 앞당겼습니다. 이를 통해 먼저 현재 규제지역에서 주택 가격에 따라 20∼50%로 차등 적용되고 있는 무주택자 대상 ltv 규제는 50%로 일원화되고, 아파트 가격 제한도 없어지게 됩니다. 현재 비규제지역에서는 무주택자를 대상으로 ltv 70%가 적용되고 있지만, 투기과열지구에서는 무주택자라도 9억 원 이하 주택은 40%, 9억 원 초과 주택은 20%의 ltv를 적용받습니다. 다만 규제지역 내 다주택자를 상대로 신규 주택대출을 금지한 규제는 지금처럼 유지됩니다. 또 총부채원리금상환비율(dsr)에 대한 규제(40%) 역시 그대로 적용됩니다.

target_text
주택담보대출,ltv 규제 완화,dsr


Row 1
input_text
송혜교x장기용 "끝까지 사랑일 거야" ..애절함 폭발 '지헤중', 순간 최고 9% [종합] (엑스포츠뉴스 조혜진 기자) '지헤중' 송혜교와 장기용의 애절 엔딩이 뜨거운 반응을 얻고 있다. 11일 방송된 sbs 금토드라마 '지금, 헤어지는 중입니다'(극본 제인, 연출 이길복, 이하 '지헤중') 10회에서는 하영은(송혜교 분)과 윤재국(장기용)이 더욱 가혹해진 시련과 마주하게 됐다. 두 사람 모두 큰 상처를 받았지만 자신보다 상대를 더 걱정하고 가슴 아파했다. 아픈 만큼 서로를 향한 사랑도, 애절함도 커진 '지헤중' 10회는 순간 최고 시청률 9%를 기록했다. (닐슨코리아 제공) 앞

In [73]:
# def clean_text(text: str) -> str:
#     sentences_cleaned = [s for sent in text for s in sent.split("\n")]
#     sentences_cleaned_no_titles = [
#         sent
#         for sent in sentences_cleaned
#         if len(sent) > 0 and sent[-1] in string.punctuation
#     ]
#     text_cleaned = "\n".join(sentences_cleaned_no_titles)
#     return text_cleaned

### Model Test

In [76]:
input_ids = tokenizer("qa question: 당신의 이름은 무엇인가요?", return_tensors='pt').input_ids
labels = tokenizer("T5 입니다.", return_tensors='pt').input_ids

input_ids, '\n', labels

outputs = model(input_ids=input_ids, labels=labels)

print(f"loss={outputs.loss} logits={outputs.logits}")

loss=7.992946624755859 logits=tensor([[[-10.2988,   6.8362,   2.2557,  ...,  -4.8973,  -5.5854,  -0.3355],
         [-12.8292,   5.7808,  10.2412,  ...,   0.6629,   0.3380,   1.9591],
         [-14.1662,   5.7439,   9.1418,  ...,  -1.4251,  -1.7599,   0.6547],
         [-12.2927,   2.8654,   6.4135,  ...,  -1.8495,  -2.1132,  -1.0649],
         [-14.7856,   8.0018,  14.4974,  ...,  -0.7229,  -0.9786,   1.7808],
         [-14.8719,   7.2765,  10.6075,  ...,  -0.4772,  -0.3803,   1.4586]]],
       grad_fn=<UnsafeViewBackward0>)


In [77]:
output = model.generate(input_ids, max_length=512)
tokenizer.batch_decode(output, skip_special_tokens=True)

[' qa qa qa question: qa']

### Sequence Length Check

In [78]:
encoded = tokenizer('하늘나라').input_ids
tokenizer.batch_decode(encoded)

['하늘', '나라', '</s>']

In [80]:
df_train.input_text.progress_apply(lambda x: len(tokenizer(x).input_ids)).describe()

100%|██████████| 1688/1688 [00:01<00:00, 943.48it/s] 


count    1688.000000
mean      424.037915
std       246.758470
min         7.000000
25%       218.750000
50%       487.000000
75%       523.250000
max      1450.000000
Name: input_text, dtype: float64

In [81]:
df_valid.input_text.progress_apply(lambda x: len(tokenizer(x).input_ids)).describe()

100%|██████████| 825/825 [00:00<00:00, 1030.40it/s]


count     825.000000
mean      450.058182
std       292.913903
min        11.000000
25%       202.000000
50%       481.000000
75%       543.000000
max      1521.000000
Name: input_text, dtype: float64

In [82]:
df_train.target_text.progress_apply(lambda x: len(tokenizer(x).input_ids)).describe()

100%|██████████| 1688/1688 [00:00<00:00, 8324.48it/s]


count    1688.000000
mean       27.263626
std        24.719454
min         2.000000
25%        13.000000
50%        18.000000
75%        29.000000
max       156.000000
Name: target_text, dtype: float64

In [83]:
df_valid.target_text.progress_apply(lambda x: len(tokenizer(x).input_ids)).describe()

100%|██████████| 825/825 [00:00<00:00, 10495.81it/s]


count    825.000000
mean      23.739394
std       21.389504
min        3.000000
25%       12.000000
50%       17.000000
75%       24.000000
max      132.000000
Name: target_text, dtype: float64

### Test Encoding

In [84]:
prefix = "key phrase generation: "

max_input_length = 512
max_target_length = 64

def preprocess_function(examples):
    inputs = prefix + examples["input_text"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [85]:
model_inputs = preprocess_function(df_train.iloc[0])

In [86]:
tokenizer.decode(model_inputs['input_ids'])

'key phrase generation: 다음 달부터 15억 초과 주택 대출 허용...ltv 50%도 다음 달부터는 무주택자의 경우 규제 지역에서도 주택담보대출비율(ltv)이 50%로 일괄 적용되고, 투기과열지구의 15억 원이 넘는 아파트에 대한 주택담보대출도 허용됩니다. 금융위원회는 최근 이 같은 내용의 대출 규제 완화 방침을 포함한 개정 은행업 감독규정을 고시하고 다음 달 1일부터 시행에 들어간다고 밝혔습니다. 앞서 정부는 지난달 제11차 비상경제민생회의에서 부동산 규제 완화 방안을 발표하고, 이달 10일 제3차 부동산 관계장관회의에서는 시행 시기를 내년 초에서 연내로 앞당겼습니다. 이를 통해 먼저 현재 규제지역에서 주택 가격에 따라 20∼50%로 차등 적용되고 있는 무주택자 대상 ltv 규제는 50%로 일원화되고, 아파트 가격 제한도 없어지게 됩니다. 현재 비규제지역에서는 무주택자를 대상으로 ltv 70%가 적용되고 있지만, 투기과열지구에서는 무주택자라도 9억 원 이하 주택은 40%, 9억 원 초과 주택은 20%의 ltv를 적용받습니다. 다만 규제지역 내 다주택자를 상대로 신규 주택대출을 금지한 규제는 지금처럼 유지됩니다. 또 총부채원리금상환비율(dsr)에 대한 규제(40%) 역시 그대로 적용됩니다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [87]:
tokenizer.decode(model_inputs['labels'])

'주택담보대출,ltv 규제 완화,dsr</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

### Save Preprocessed Data

In [74]:
# df_train.to_json(
#     "./data/prepared/train_set.json",
#     orient="records",
#     lines=True,
#     force_ascii=False,
# )
# df_valid.to_json(
#     "./data/prepared/valid_set.json",
#     orient="records",
#     lines=True,
#     force_ascii=False,
# )

In [88]:
df_data = pd.concat([df_train, df_valid])

In [147]:
df_data.target_text.apply(lambda x: len(x.split(','))).describe()

count    2513.000000
mean        6.176283
std         4.790471
min         1.000000
25%         3.000000
50%         4.000000
75%         7.000000
max        32.000000
Name: target_text, dtype: float64

In [89]:
# df_data.to_pickle('./data/total_2513_samples.pickle')

### Check Data

In [95]:
checker = df_data.target_text.apply(lambda x: len(tokenizer.encode(x))) > 64
checked = df_data[checker]

for idx, row in checked.iterrows():
    print(row.target_text)
    print()

thailand,oil,spill,cleanup,operation,pipeline,coast,rayong,province,gulf,thailand,undersea,hose,tankers,offshore,mooring,point,star,petroleum,refining,co.

교육부,전면 등교 준비,코로나19 방역,대통령 직속 국가교육회의,국민 참여 교육과정 개정,사회적 협의 추진계획,온라인 설문조사,권역별 토론회,청년·청소년 토론회,의견 수렴 결과,2022 개정 교육과정,국민 의견 반영

일잘러,회사 생활,커뮤니케이션,협상,부자의 말센스,김주하 작가,동기부여,리더,직원들의 의욕,수평적 문화,관리자들,대화방법,자존감,2030 직장인들,협상에서 성공,창업,경험 부족,좋은 사람

브래드 피트,냄새,윤여정,아카데미 시상식,여우조연상,무례한 질문,농담,인종차별적 태도,유머 감각,수상 소감,아시아인 이름 발음,서구인 태도,다양성의 가치,차별과 혐오,영국 아카데미 시상식

GS건설,봉담자이 라피네,견본주택 오픈,경기도 화성시,동화지구,아파트,전용면적,주거타운,광역 교통망,수도권 제2외곽순환고속도로,봉담 IC,과천-봉담 고속도로,광역버스 노선,마을버스 다수 노선,수인분당선 오목천역

빅뱅 태양,컴백 암시,34번째 생일,팬들,고마움,인스타그램 스토리,메시지,생일 축하 편지,추억들,사랑,향후 활동,기대감,건강하게,코첼라 밸리 뮤직 앤드 아츠 페스티벌,코로나 19 여파,지드래곤,신곡 작업,

유흥업소 점주들,경찰 대치,20일 오후,한국유흥음식업중앙회,한국단란주점업중앙회,한국콜라텍협회,관계자 및 업주들,정부,집합금지 해제,손실보상법 통과,서울 중구,서울시청 내부,진입,경찰

희귀병치료제,환자,그림에 떡,맞춤형 보험 급여,희귀질환,유전자 치료,면역세포 치료,혁신기술,완치 기대,미국 식품의약국,임상시험,건강보험 적용,접근성 강화,국회보건복지위원회,강선우 의원,한국희귀난치성질환연합회,유튜브 토론회,강혜영 교수,혁신신약 개발,보험 등재,심사 기간 단축

하나금융그룹,ESG 캠페인 광고

In [91]:
checker = df_data.input_text.str.contains('인천항')
df_data[checker].input_text.iloc[0]

'인천항 선박서 부상자 구조하는 소방대원들 (인천=연합뉴스) 31일 오후 인천시 중구 항동7가 2부두에서 정박 중인 선박에서 작업 중 다리를 다친 a(38)씨를 소방당국이 구조하고 있다.'